# Train PhoBERT NER trên GPU RTX 4050

Training PhoBERT cho Named Entity Recognition với GPU acceleration.

In [5]:
import torch
import json
import numpy as np
from transformers import (
    AutoTokenizer,
    AutoModelForTokenClassification,
    TrainingArguments,
    Trainer,
    DataCollatorForTokenClassification
)
from datasets import Dataset
from seqeval.metrics import f1_score, precision_score, recall_score, classification_report

print("="*60)
print("🔍 GPU Status")
print("="*60)
print(f"PyTorch: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"VRAM: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.2f} GB")
    device = "cuda"
else:
    print("⚠️ No GPU detected, using CPU")
    device = "cpu"
print("="*60)

🔍 GPU Status
PyTorch: 2.6.0+cu124
CUDA available: True
GPU: NVIDIA GeForce RTX 4050 Laptop GPU
VRAM: 6.00 GB


## 1. Load dữ liệu

In [6]:
# Load JSON data
print("📂 Loading data...")

with open("../../data/processed/train_phobert.json", "r", encoding="utf-8") as f:
    train_data = json.load(f)

with open("../../data/processed/val_phobert.json", "r", encoding="utf-8") as f:
    val_data = json.load(f)

print(f"✓ Train: {len(train_data)} sentences")
print(f"✓ Val: {len(val_data)} sentences")

# Sample
print(f"\n📝 Sample:")
print(f"Tokens: {train_data[0]['tokens'][:5]}...")
print(f"Tags: {train_data[0]['ner_tags'][:5]}...")

📂 Loading data...
✓ Train: 800 sentences
✓ Val: 201 sentences

📝 Sample:
Tokens: ['Có', 'phải', 'Sốt', 'đôi', 'khi']...
Tags: ['O', 'O', 'B-SYMPTOM', 'I-SYMPTOM', 'I-SYMPTOM']...


## 2. Tạo label mapping

In [7]:
# Get all unique labels
all_labels = set()
for item in train_data + val_data:
    all_labels.update(item['ner_tags'])

label_list = sorted(list(all_labels))
label2id = {label: i for i, label in enumerate(label_list)}
id2label = {i: label for label, i in label2id.items()}

print(f"🏷️  Labels ({len(label_list)}): {label_list}")
print(f"\n📊 Label mapping:")
for label, idx in label2id.items():
    print(f"  {idx}: {label}")

🏷️  Labels (8): ['B-DISEASE', 'B-SYMPTOM', 'I-DISEASE', 'I-I-SYMPTOM', 'I-SYMPTEM', 'I-SYMPTOM', 'O', 'Tôi']

📊 Label mapping:
  0: B-DISEASE
  1: B-SYMPTOM
  2: I-DISEASE
  3: I-I-SYMPTOM
  4: I-SYMPTEM
  5: I-SYMPTOM
  6: O
  7: Tôi


## 3. Convert labels sang IDs

In [8]:
# Convert labels to IDs
def convert_labels_to_ids(data, label2id):
    converted = []
    for item in data:
        converted.append({
            "tokens": item["tokens"],
            "ner_tags": [label2id[label] for label in item["ner_tags"]]
        })
    return converted

train_data = convert_labels_to_ids(train_data, label2id)
val_data = convert_labels_to_ids(val_data, label2id)

print("✓ Converted labels to IDs")
print(f"\nSample after conversion:")
print(f"Tokens: {train_data[0]['tokens'][:5]}")
print(f"Tag IDs: {train_data[0]['ner_tags'][:5]}")

✓ Converted labels to IDs

Sample after conversion:
Tokens: ['Có', 'phải', 'Sốt', 'đôi', 'khi']
Tag IDs: [6, 6, 1, 5, 5]


## 4. Tạo Hugging Face Dataset

In [9]:
# Convert to HF Dataset
train_dataset = Dataset.from_list(train_data)
val_dataset = Dataset.from_list(val_data)

print(f"✓ Created HF Datasets")
print(f"\n📊 Dataset info:")
print(train_dataset)

✓ Created HF Datasets

📊 Dataset info:
Dataset({
    features: ['tokens', 'ner_tags'],
    num_rows: 800
})


## 5. Load PhoBERT

In [10]:
model_checkpoint = "vinai/phobert-base"

print(f"📥 Loading {model_checkpoint}...")

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, use_fast=True)
model = AutoModelForTokenClassification.from_pretrained(
    model_checkpoint,
    num_labels=len(label_list),
    id2label=id2label,
    label2id=label2id
)

# Move to GPU
model = model.to(device)

print(f"\n✅ Model loaded on {device}")
print(f"Parameters: {model.num_parameters():,}")

📥 Loading vinai/phobert-base...


Some weights of RobertaForTokenClassification were not initialized from the model checkpoint at vinai/phobert-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.



✅ Model loaded on cuda
Parameters: 134,413,832


## 6. Tokenize và align labels

In [11]:
def tokenize_and_align_labels(examples):
    """
    Tokenize text và align labels với subword tokens
    """
    tokenized_inputs = tokenizer(
        examples["tokens"],
        truncation=True,
        is_split_into_words=True,
        max_length=256,
        padding=False  # Không padding ở đây, để collator làm
    )
    
    labels = []
    
    for i, label in enumerate(examples["ner_tags"]):
        # PhoBERT tokenizer không có word_ids() → phải tự align
        word_ids = []
        current_word_idx = 0
        
        # Lấy input_ids của câu này
        input_ids = tokenized_inputs["input_ids"][i]
        tokens_from_ids = tokenizer.convert_ids_to_tokens(input_ids)
        
        for idx, token in enumerate(tokens_from_ids):
            # Special tokens
            if token in ["<s>", "</s>", "<pad>"]:
                word_ids.append(None)
            # Subword token (bắt đầu với @@)
            elif token.startswith("@@"):
                word_ids.append(current_word_idx - 1)  # Thuộc từ trước đó
            # Normal token
            else:
                word_ids.append(current_word_idx)
                current_word_idx += 1
        
        # Align labels
        label_ids = []
        previous_word_idx = None
        
        for word_idx in word_ids:
            # Special token → -100
            if word_idx is None:
                label_ids.append(-100)
            # Từ mới → gán label
            elif word_idx != previous_word_idx:
                if word_idx < len(label):
                    label_ids.append(label[word_idx])
                else:
                    label_ids.append(-100)
            # Subword của từ cũ → -100
            else:
                label_ids.append(-100)
            
            previous_word_idx = word_idx
        
        labels.append(label_ids)
    
    tokenized_inputs["labels"] = labels
    return tokenized_inputs

print("🔄 Tokenizing...")
tokenized_train = train_dataset.map(tokenize_and_align_labels, batched=True)
tokenized_val = val_dataset.map(tokenize_and_align_labels, batched=True)

print("✓ Tokenization complete")

🔄 Tokenizing...


Map: 100%|██████████| 800/800 [00:00<00:00, 5986.31 examples/s]

Map: 100%|██████████| 201/201 [00:00<00:00, 6564.88 examples/s]

✓ Tokenization complete


## 7. Data Collator

In [12]:
data_collator = DataCollatorForTokenClassification(
    tokenizer=tokenizer,
    padding=True
)

print("✓ Data collator ready")

✓ Data collator ready


## 8. Metrics

In [13]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=2)
    
    true_labels = []
    pred_labels = []
    
    for prediction, label in zip(predictions, labels):
        true_label = []
        pred_label = []
        
        for pred, lab in zip(prediction, label):
            if lab != -100:
                true_label.append(id2label[lab])
                pred_label.append(id2label[pred])
        
        true_labels.append(true_label)
        pred_labels.append(pred_label)
    
    precision = precision_score(true_labels, pred_labels)
    recall = recall_score(true_labels, pred_labels)
    f1 = f1_score(true_labels, pred_labels)
    
    return {
        "precision": precision,
        "recall": recall,
        "f1": f1
    }

print("✓ Metrics function ready")

✓ Metrics function ready


## 9. Training Arguments - Tối ưu cho RTX 4050 (6GB)

In [14]:
import os

training_args = TrainingArguments(
    output_dir="../../models/phobert_ner_checkpoints",
    
    # ===== GPU OPTIMIZATION =====
    per_device_train_batch_size=16,      # RTX 4050 6GB có thể handle
    per_device_eval_batch_size=16,
    gradient_accumulation_steps=2,        # Effective batch = 16 * 2 = 32
    fp16=True,                            # Mixed Precision (tiết kiệm VRAM)
    dataloader_num_workers=4,             # Tăng tốc data loading
    dataloader_pin_memory=True,
    
    # ===== TRAINING PARAMS =====
    num_train_epochs=5,
    learning_rate=2e-5,
    weight_decay=0.01,
    warmup_ratio=0.1,
    max_grad_norm=1.0,
    
    # ===== EVALUATION ===== (ĐÃ SỬA)
    eval_strategy="epoch",                # Đổi từ evaluation_strategy
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    greater_is_better=True,
    
    # ===== LOGGING =====
    logging_dir="../../models/logs",
    logging_steps=20,
    logging_first_step=True,
    
    # ===== SAVING =====
    save_total_limit=2,  # Chỉ giữ 2 checkpoint tốt nhất
    
    # ===== OTHER =====
    seed=42,
    report_to="none",
)

print("✅ Training Arguments:")
print(f"   Device: {training_args.device}")
print(f"   FP16: {training_args.fp16}")
print(f"   Batch size: {training_args.per_device_train_batch_size}")
print(f"   Gradient accumulation: {training_args.gradient_accumulation_steps}")
print(f"   Effective batch: {training_args.per_device_train_batch_size * training_args.gradient_accumulation_steps}")
print(f"   Epochs: {training_args.num_train_epochs}")

✅ Training Arguments:
   Device: cuda:0
   FP16: True
   Batch size: 16
   Gradient accumulation: 2
   Effective batch: 32
   Epochs: 5


## 10. Trainer

In [15]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_val,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

print("✅ Trainer ready")

C:\Users\Admin\AppData\Local\Temp\ipykernel_30732\2403826709.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


✅ Trainer ready


## 11. START TRAINING 🚀

In [16]:
print("\n" + "="*60)
print("🚀 BẮT ĐẦU TRAINING")
print("="*60)

# Clear GPU cache
if torch.cuda.is_available():
    torch.cuda.empty_cache()
    print("🧹 GPU cache cleared")

# Train
train_result = trainer.train()

print("\n" + "="*60)
print("✅ TRAINING COMPLETE")
print("="*60)
print(f"Loss: {train_result.training_loss:.4f}")
print(f"Time: {train_result.metrics['train_runtime']:.2f}s ({train_result.metrics['train_runtime']/60:.1f} mins)")


🚀 BẮT ĐẦU TRAINING
🧹 GPU cache cleared


Epoch,Training Loss,Validation Loss,Precision,Recall,F1
1,1.638200,0.702997,0.657471,0.718593,0.686675
2,0.639500,0.255503,0.835381,0.854271,0.844720
3,0.285600,0.211509,0.851852,0.866834,0.859278
4,0.191700,0.202463,0.856079,0.866834,0.861423
5,0.203200,0.193566,0.862843,0.869347,0.866083


e:\CHATBOT MODEL\training\venv\Lib\site-packages\seqeval\metrics\sequence_labeling.py:171: UserWarning: Tôi seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
e:\CHATBOT MODEL\training\venv\Lib\site-packages\seqeval\metrics\sequence_labeling.py:171: UserWarning: Tôi seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
e:\CHATBOT MODEL\training\venv\Lib\site-packages\seqeval\metrics\sequence_labeling.py:171: UserWarning: Tôi seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
e:\CHATBOT MODEL\training\venv\Lib\site-packages\seqeval\metrics\sequence_labeling.py:171: UserWarning: Tôi seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
e:\CHATBOT MODEL\training\venv\Lib\site-packages\seqeval\metrics\sequence_labeling.py:171: UserWarning: Tôi seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
e:\CHATBOT MODEL\training\venv\Lib\site-


✅ TRAINING COMPLETE
Loss: 0.5220
Time: 321.66s (5.4 mins)


## 12. Evaluate

In [17]:
print("\n📊 Evaluating...")
eval_results = trainer.evaluate()

print("\n✅ Results:")
print(f"   Precision: {eval_results['eval_precision']:.4f}")
print(f"   Recall: {eval_results['eval_recall']:.4f}")
print(f"   F1-score: {eval_results['eval_f1']:.4f}")


📊 Evaluating...



✅ Results:
   Precision: 0.8628
   Recall: 0.8693
   F1-score: 0.8661


e:\CHATBOT MODEL\training\venv\Lib\site-packages\seqeval\metrics\sequence_labeling.py:171: UserWarning: Tôi seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))


## 13. Save Model

In [18]:
print("\n💾 Saving model...")

output_dir = "../../models/phobert_ner_model"
os.makedirs(output_dir, exist_ok=True)

# Save model
trainer.save_model(output_dir)
tokenizer.save_pretrained(output_dir)

# Save label mapping
label_info = {
    "label2id": label2id,
    "id2label": {str(k): v for k, v in id2label.items()}
}

with open(f"{output_dir}/label_mapping.json", "w", encoding="utf-8") as f:
    json.dump(label_info, f, ensure_ascii=False, indent=2)

print(f"✅ Saved to: {output_dir}")

# Clear GPU
if torch.cuda.is_available():
    torch.cuda.empty_cache()
    print("🧹 GPU cache cleared")


💾 Saving model...
✅ Saved to: ../../models/phobert_ner_model
🧹 GPU cache cleared
✅ Saved to: ../../models/phobert_ner_model
🧹 GPU cache cleared


## 14. Test

In [20]:
print("\n🧪 TEST")
print("="*60)

test_sentence = "Tôi bị sốt cao, đau đầu và ho nhiều"

# Tokenize
inputs = tokenizer(test_sentence, return_tensors="pt").to(device)

# Predict
with torch.no_grad():
    outputs = model(**inputs)
    predictions = torch.argmax(outputs.logits, dim=-1)

# Decode
tokens = tokenizer.convert_ids_to_tokens(inputs["input_ids"][0])
predicted_labels = [id2label[p.item()] for p in predictions[0]]

print(f"\nCâu test: {test_sentence}")

# Debug: Xem cách PhoBERT tokenize
print("\n" + "="*60)
print("DEBUG: Cách PhoBERT tokenize câu")
print("="*60)
print(f"Input text: {test_sentence}")
print(f"Tokens từ tokenizer.tokenize(): {tokenizer.tokenize(test_sentence)}")
print(f"\nTokens từ convert_ids_to_tokens(): {tokens}")

print("\n" + "="*60)
print("KẾT QUẢ (Tokens thô từ PhoBERT):")
print("="*60)
for token, label in zip(tokens, predicted_labels):
    if token not in ['<s>', '</s>', '<pad>']:
        print(f"  {token:20s} -> {label}")

# ===== DECODE ĐÚNG: Ghép subword lại =====
def decode_tokens_and_labels(tokens, labels):
    """Ghép subword tokens thành từ gốc (PhoBERT style)"""
    words = []
    word_labels = []
    current_word = ""
    current_label = None
    
    for token, label in zip(tokens, labels):
        # Bỏ qua special tokens
        if token in ['<s>', '</s>', '<pad>']:
            continue
        
        # QUAN TRỌNG: PhoBERT đánh dấu @@ Ở CUỐI token (không phải đầu)
        # Ví dụ: "cao@@" nghĩa là "cao" + subword tiếp theo
        if token.endswith('@@'):
            # Lưu từ trước (nếu có)
            if current_word:
                words.append(current_word)
                word_labels.append(current_label)
            # Token này kết thúc bằng @@ → từ chưa hoàn chỉnh
            current_word = token[:-2]  # Bỏ @@
            current_label = label
        # Token bắt đầu với @@ (tiếp tục từ trước)
        elif token.startswith('@@'):
            current_word += token[2:]  # Bỏ @@
        # Token thường
        else:
            # Nếu có từ trước chưa hoàn chỉnh → ghép tiếp
            if current_word and current_word != "":
                current_word += token
            # Nếu không → bắt đầu từ mới
            else:
                # Lưu từ trước (nếu có)
                if current_word:
                    words.append(current_word)
                    word_labels.append(current_label)
                current_word = token
                current_label = label
    
    # Lưu từ cuối
    if current_word:
        words.append(current_word)
        word_labels.append(current_label)
    
    return words, word_labels

words, word_labels = decode_tokens_and_labels(tokens, predicted_labels)

print("\n" + "="*60)
print("KẾT QUẢ (Đã ghép subword - ĐÚNG):")
print("="*60)
for word, label in zip(words, word_labels):
    print(f"  {word:20s} -> {label}")

print("\n✅ Hoàn tất!")
print("\n📘 Giải thích PhoBERT tokenization:")
print("- PhoBERT có thể đánh dấu @@ ở 2 vị trí:")
print("  + Cuối token: 'cao@@' → từ chưa hoàn chỉnh, có subword tiếp theo")
print("  + Đầu token: '@@t' → subword tiếp theo của từ trước")
print("- Ví dụ: 'information' → ['in', '@@for', '@@mation']")
print("- Hoặc: 'cao nhất' → ['cao@@', 'nhất'] (trong một số trường hợp)")


🧪 TEST

Câu test: Tôi bị sốt cao, đau đầu và ho nhiều

DEBUG: Cách PhoBERT tokenize câu
Input text: Tôi bị sốt cao, đau đầu và ho nhiều
Tokens từ tokenizer.tokenize(): ['Tôi', 'bị', 'sốt', 'cao@@', ',', 'đau', 'đầu', 'và', 'ho', 'nhiều']

Tokens từ convert_ids_to_tokens(): ['<s>', 'Tôi', 'bị', 'sốt', 'cao@@', ',', 'đau', 'đầu', 'và', 'ho', 'nhiều', '</s>']

KẾT QUẢ (Tokens thô từ PhoBERT):
  Tôi                  -> O
  bị                   -> O
  sốt                  -> B-SYMPTOM
  cao@@                -> I-SYMPTOM
  ,                    -> O
  đau                  -> B-SYMPTOM
  đầu                  -> I-SYMPTOM
  và                   -> O
  ho                   -> B-SYMPTOM
  nhiều                -> I-SYMPTOM

KẾT QUẢ (Đã ghép subword - ĐÚNG):
  Tôibịsốt             -> O
  cao,đauđầuvàhonhiều  -> I-SYMPTOM

✅ Hoàn tất!

📘 Giải thích PhoBERT tokenization:
- PhoBERT có thể đánh dấu @@ ở 2 vị trí:
  + Cuối token: 'cao@@' → từ chưa hoàn chỉnh, có subword tiếp theo
  + Đầu token: '@@t' →